In [7]:
import pandas as pd 
import numpy as np 
import random

np.random.seed(42)
random.seed(42)

In [11]:
# 导入模型训练数据
train_path = "../kaggle/input/xunfei-dataset/train.csv"
train_df = pd.read_csv(train_path)
print(f"len(train_df):{(len(train_df))}")
train_df.head()

len(train_df):620356


,uuid,eid,udmap,common_ts,x1,x2,x3,x4,x5,x6,x7,x8,target
0,0,26,"{""key3"":""67804"",""key2"":""650""}",1689673468244,4,0,41,107,206,1,0,1,0
1,1,26,"{""key3"":""67804"",""key2"":""484""}",1689082941469,4,0,41,24,283,4,8,1,0
2,2,8,unknown,1689407393040,4,0,41,71,288,4,7,1,0
3,3,11,unknown,1689467815688,1,3,41,17,366,1,6,1,0
4,4,26,"{""key3"":""67804"",""key2"":""650""}",1689491751442,0,3,41,92,383,4,8,1,0


In [12]:
# 导入测试数据
test_path = "../kaggle/input/xunfei-dataset/test.csv"
test_df = pd.read_csv(test_path)
print(f"len(test_df):{(len(test_df))}")
test_df.head()

len(test_df):206785


,uuid,eid,udmap,common_ts,x1,x2,x3,x4,x5,x6,x7,x8
0,0,11,unknown,1689594441029,4,1,41,85,343,4,8,1
1,1,35,unknown,1689551329947,4,0,41,24,283,1,6,1
2,2,34,"{""key3"":""73457"",""key2"":""936""}",1688965066999,4,2,41,71,288,4,2,0
3,3,0,"{""key3"":""18771""}",1689308623754,1,0,41,104,37,4,8,1
4,4,26,"{""key3"":""67804"",""key2"":""650""}",1689516018904,0,1,41,115,217,4,8,1


In [13]:
# 整合所有数据，按照之前建模的操作处理

total_df = pd.concat((train_df,test_df), axis = 0)

total_df.drop(['uuid','udmap'], axis = 1, inplace = True)

print(f"len(total_df):{len(total_df)}")
total_df.head()

len(total_df):827141


,eid,common_ts,x1,x2,x3,x4,x5,x6,x7,x8,target
0,26,1689673468244,4,0,41,107,206,1,0,1,0.0
1,26,1689082941469,4,0,41,24,283,4,8,1,0.0
2,8,1689407393040,4,0,41,71,288,4,7,1,0.0
3,11,1689467815688,1,3,41,17,366,1,6,1,0.0
4,26,1689491751442,0,3,41,92,383,4,8,1,0.0


In [14]:
# 计算相对于2023年（非闰年）的进度百分比
total_df['common_ts'] = ((total_df['common_ts'] / 1000) % 31536000) / 31536000 
total_df.head()

,eid,common_ts,x1,x2,x3,x4,x5,x6,x7,x8,target
0,26,0.579194,4,0,41,107,206,1,0,1,0.0
1,26,0.560469,4,0,41,24,283,4,8,1,0.0
2,8,0.570757,4,0,41,71,288,4,7,1,0.0
3,11,0.572673,1,3,41,17,366,1,6,1,0.0
4,26,0.573432,0,3,41,92,383,4,8,1,0.0


In [15]:
# 选取相关数据，以供后续训练
# 皮尔逊相关系数(+1正相关,-1负相关,接近没有明显相关性)
pearson = total_df.corr(method = 'pearson').values[-1]
choose1 = np.where(abs(pearson) >= 0.01)[0]
print(f"len(choose1):{len(choose1)},choose1:{choose1}")
choose = total_df.keys().values[choose1]
total_df = total_df[choose]
total_df.head()

len(choose1):9,choose1:[ 0  1  3  5  6  7  8  9 10]


,eid,common_ts,x2,x4,x5,x6,x7,x8,target
0,26,0.579194,0,107,206,1,0,1,0.0
1,26,0.560469,0,24,283,4,8,1,0.0
2,8,0.570757,0,71,288,4,7,1,0.0
3,11,0.572673,3,17,366,1,6,1,0.0
4,26,0.573432,3,92,383,4,8,1,0.0


In [16]:
# 得到'common_ts'列的均值和方差
total_df.describe()

,eid,common_ts,x2,x4,x5,x6,x7,x8,target
count,827141.000000,827141.000000,827141.000000,827141.000000,827141.000000,827141.000000,827141.000000,827141.000000,620356.000000
mean,22.150853,0.567872,1.105287,82.899570,224.947866,2.902127,5.864469,0.855634,0.140566
std,12.139231,0.008717,1.173478,44.115095,114.293439,1.444678,2.576408,0.351461,0.347574
min,0.000000,0.538215,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,11.000000,0.560602,0.000000,51.000000,133.000000,1.000000,6.000000,1.000000,0.000000
50%,26.000000,0.569755,1.000000,86.000000,241.000000,4.000000,7.000000,1.000000,0.000000
75%,34.000000,0.575678,2.000000,107.000000,313.000000,4.000000,7.000000,1.000000,0.000000
max,42.000000,0.579908,3.000000,151.000000,413.000000,4.000000,9.000000,1.000000,1.000000


In [17]:
np.unique(total_df['eid'].values)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42], dtype=int64)

In [18]:
# 将eid处理后作为新特征传入

eid_target = train_df['target'].groupby([train_df['eid']]).mean()
eid = eid_target.keys().values
target = eid_target.values
eid_target = pd.DataFrame({"eid":eid,"eid_target":target})
eid_target.head()

,eid,eid_target
0,0,0.069281
1,1,0.485054
2,2,0.139414
3,3,0.352640
4,4,0.486146


In [19]:
total_df = pd.merge(total_df, eid_target, on = "eid", how = "left")
total_df.head()

,eid,common_ts,x2,x4,x5,x6,x7,x8,target,eid_target
0,26,0.579194,0,107,206,1,0,1,0.0,0.072707
1,26,0.560469,0,24,283,4,8,1,0.0,0.072707
2,8,0.570757,0,71,288,4,7,1,0.0,0.097401
3,11,0.572673,3,17,366,1,6,1,0.0,0.098421
4,26,0.573432,3,92,383,4,8,1,0.0,0.072707


In [20]:
# 用具有周期性的三角函数新增特征（用前面得到的均值和方差）
# 归一化
total_df['sin_norm'] = np.sin(2 * np.pi * (total_df['common_ts'] - 0.567872) / 0.008717)
total_df['cos_norm'] = np.cos(2 * np.pi * (total_df['common_ts'] - 0.567872) / 0.008717)
# 未归一化
total_df['sin'] = np.sin(2 * np.pi * total_df['common_ts']) 
total_df['cos'] = np.cos(2 * np.pi * total_df['common_ts'])

In [21]:
train_df = total_df[:len(train_df)]
test_df = total_df[len(train_df):]

In [22]:
y = train_df['target'].values
X = train_df.drop(['target'], axis = 1).values

In [27]:
# 划分训练集和测试集的函数
def train_test_split(dataX,datay,shuffle=True,percentage=0.8):
    """
    将训练数据X和标签y以numpy.array数组的形式传入
    划分的比例定为 训练集:测试集 = 8:2 
    """
    if shuffle :
        random_num=[index for index in range(len(dataX))]
        np.random.shuffle(random_num)
        dataX=dataX[random_num]
        datay=datay[random_num]

    split_num = int(len(dataX) * percentage)
    train_X = dataX[:split_num]
    train_y = datay[:split_num]
    test_X = dataX[split_num:]
    test_y = datay[split_num:]
    return train_X,train_y,test_X,test_y

In [28]:
train_X,train_y,valid_X,valid_y = train_test_split(X,y,percentage=0.9)
print(f"train_X.shape:{train_X.shape},valid_X.shape:{valid_X.shape}")

train_X.shape:(558320, 13),valid_X.shape:(62036, 13)


In [24]:
!pip install FLAML
!pip install "ray[tune]<2.5.0"


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/294.9 kB ? eta -:--:--
     -------------------------------------- 294.9/294.9 kB 9.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/21.7 MB ? eta -:--:--
      --------------------------------------- 0.5/21.7 MB 14.2 MB/s eta 0:00:02
     -- ------------------------------------- 1.4/21.7 MB 18.1 MB/s eta 0:00:02
     ---- ----------------------------------- 2.3/21.7 MB 18.5 MB/s eta 0:00:02
     ------- -------------------------------- 4.1/21.7 MB 24.1 MB/s eta 0:00:01
     ---------- ----------------------------- 5.4/21.7 MB 24.8 MB/s eta 0:00:01
     ------------- -------------------------- 7.2/21.7 MB 27.1 MB/s eta 0:00:01
     ---------------- ----------------------- 9.1/21.7 MB 29.1 MB/s eta 0:00:01
     -------------------- ------------------ 11.4/21.7 MB 34.4 MB/s eta 0:00:01
     --------

In [25]:
from flaml import AutoML
automl = AutoML()

In [30]:
automl.fit(train_X, train_y, task = "classification", metric = 'accuracy', time_budget = 3600)

[flaml.automl.logger: 08-24 22:17:58] {1679} INFO - task = classification
[flaml.automl.logger: 08-24 22:17:58] {1690} INFO - Evaluation method: holdout
[flaml.automl.logger: 08-24 22:17:58] {1788} INFO - Minimizing error metric: 1-accuracy
[flaml.automl.logger: 08-24 22:17:58] {1900} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
[flaml.automl.logger: 08-24 22:17:58] {2218} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 08-24 22:17:58] {2344} INFO - Estimated sufficient time budget=29515s. Estimated necessary time budget=680s.
[flaml.automl.logger: 08-24 22:17:58] {2391} INFO -  at 0.5s,	estimator lgbm's best error=0.1406,	best estimator lgbm's best error=0.1406
[flaml.automl.logger: 08-24 22:17:58] {2218} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 08-24 22:17:59] {2391} INFO -  at 0.6s,	estimator lgbm's best error=0.1406,	best estimator lgbm's best error=0.1406
[flaml.automl.logger: 08-24

f:\Anaconda\envs\mcm\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[flaml.automl.logger: 08-24 22:17:59] {2391} INFO -  at 1.4s,	estimator extra_tree's best error=0.1406,	best estimator lgbm's best error=0.1406
[flaml.automl.logger: 08-24 22:17:59] {2218} INFO - iteration 5, current learner lgbm
[flaml.automl.logger: 08-24 22:18:00] {2391} INFO -  at 1.5s,	estimator lgbm's best error=0.1365,	best estimator lgbm's best error=0.1365
[flaml.automl.logger: 08-24 22:18:00] {2218} INFO - iteration 6, current learner xgboost
[flaml.automl.logger: 08-24 22:18:00] {2391} INFO -  at 1.6s,	estimator xgboost's best error=0.1406,	best estimator lgbm's best error=0.1365
[flaml.automl.logger: 08-24 22:18:00] {2218} INFO - iteration 7, current learner lgbm
[flaml.automl.logger: 08-24 22:18:00] {2391} INFO -  at 1.7s,	estimator lgbm's best error=0.1340,	best estimator lgbm's best error=0.1340


f:\Anaconda\envs\mcm\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[flaml.automl.logger: 08-24 22:18:00] {2218} INFO - iteration 8, current learner lgbm
[flaml.automl.logger: 08-24 22:18:00] {2391} INFO -  at 1.9s,	estimator lgbm's best error=0.1340,	best estimator lgbm's best error=0.1340
[flaml.automl.logger: 08-24 22:18:00] {2218} INFO - iteration 9, current learner lgbm
[flaml.automl.logger: 08-24 22:18:00] {2391} INFO -  at 2.0s,	estimator lgbm's best error=0.1340,	best estimator lgbm's best error=0.1340
[flaml.automl.logger: 08-24 22:18:00] {2218} INFO - iteration 10, current learner xgboost
[flaml.automl.logger: 08-24 22:18:00] {2391} INFO -  at 2.0s,	estimator xgboost's best error=0.1406,	best estimator lgbm's best error=0.1340
[flaml.automl.logger: 08-24 22:18:00] {2218} INFO - iteration 11, current learner xgboost
[flaml.automl.logger: 08-24 22:18:00] {2391} INFO -  at 2.1s,	estimator xgboost's best error=0.1406,	best estimator lgbm's best error=0.1340
[flaml.automl.logger: 08-24 22:18:00] {2218} INFO - iteration 12, current learner extra_tr

...

f:\Anaconda\envs\mcm\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[flaml.automl.logger: 08-24 23:00:57] {2391} INFO -  at 2579.0s,	estimator rf's best error=0.1271,	best estimator lgbm's best error=0.0438
[flaml.automl.logger: 08-24 23:00:57] {2218} INFO - iteration 151, current learner rf
[flaml.automl.logger: 08-24 23:00:59] {2391} INFO -  at 2580.8s,	estimator rf's best error=0.1271,	best estimator lgbm's best error=0.0438
[flaml.automl.logger: 08-24 23:00:59] {2218} INFO - iteration 152, current learner xgboost
[flaml.automl.logger: 08-24 23:00:59] {2391} INFO -  at 2580.9s,	estimator xgboost's best error=0.1350,	best estimator lgbm's best error=0.0438
[flaml.automl.logger: 08-24 23:00:59] {2218} INFO - iteration 153, current learner xgboost


f:\Anaconda\envs\mcm\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
f:\Anaconda\envs\mcm\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[flaml.automl.logger: 08-24 23:00:59] {2391} INFO -  at 2581.1s,	estimator xgboost's best error=0.1349,	best estimator lgbm's best error=0.0438
[flaml.automl.logger: 08-24 23:00:59] {2218} INFO - iteration 154, current learner extra_tree
[flaml.automl.logger: 08-24 23:01:01] {2391} INFO -  at 2582.6s,	estimator extra_tree's best error=0.1212,	best estimator lgbm's best error=0.0438
[flaml.automl.logger: 08-24 23:01:01] {2218} INFO - iteration 155, current learner lgbm
[flaml.automl.logger: 08-24 23:19:45] {2391} INFO -  at 3707.1s,	estimator lgbm's best error=0.0434,	best estimator lgbm's best error=0.0434


KeyboardInterrupt: 

In [1]:
#二分类任务的召回率
def recall(y_true,y_pred):
    #Positive(类别为1)，True:预测正确，即预测值为1
    TP = np.sum(y_true + y_pred == 2)
    #Negative(类别为0),True:预测正确,即预测值为0
    TN = np.sum(y_true + y_pred == 0)
    #False表示预测错误，预测为1，实际是0
    FP = np.sum(np.where((y_true == 0) & (y_pred == 1),1,0))
    #False表示预测错误，预测为0,实际为1
    FN = np.sum(np.where((y_true == 1) & (y_pred == 0),1,0))
    #召回率的计算:
    Recall_rate = TP / (TP + FN)
    return Recall_rate

In [2]:
def accuracy(y_true,y_pred):
    return np.sum(y_true == y_pred) / len(y_true)

In [3]:
def f1_score(y_true,y_pred):
    Recall_rate = recall(y_true,y_pred)
    Acc = accuracy(y_true,y_pred)
    return 2 * Acc * Recall_rate / (Acc + Recall_rate)

In [4]:
train_pred = automl.predict(train_X)
print(f"train_accuracy:{accuracy(train_y,train_pred)},train_f1_score:{f1_score(train_y,train_pred)}")
valid_pred = automl.predict(valid_X)
print(f"valid_accuracy:{accuracy(valid_y,valid_pred)},valid_f1_score:{f1_score(valid_y,valid_pred)}")

NameError: name 'automl' is not defined